In [104]:
import pandas as pd
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics
import pickle
# import graphviz

le = preprocessing.LabelEncoder()

# Get data, separate into features and labels
with open('./data/testing_data.obj', 'rb') as testing_file:
    testing_data = pickle.load(testing_file)

with open('./data/training_data.obj', 'rb') as training_file:
    training_data = pickle.load(training_file)

tr_data_df = pd.DataFrame.from_records(training_data['tweets'], columns=['Party', 'Tweet_Text', 'Hashtags', 'Mentions', 'Retweet', 'Handle', 'Full_Text'])

test_data_df = pd.DataFrame.from_records(testing_data['tweets'], columns=['Party', 'Tweet_Text', 'Hashtags', 'Mentions', 'Retweet', 'Handle', 'Full_Text'])

tr_encoded = tr_data_df.copy()
tr_encoded["Party"] = le.fit_transform(tr_encoded["Party"])
tr_encoded["Tweet_Text"] = le.fit_transform(tr_encoded["Tweet_Text"])
#tr_encoded["Hashtags"] = le.fit_transform(tr_encoded["Hashtags"])
#tr_encoded["Mentions"] = le.fit_transform(tr_encoded["Mentions"])
tr_encoded["Retweet"] = le.fit_transform(tr_encoded["Retweet"])
tr_encoded["Handle"] = le.fit_transform(tr_encoded["Handle"])
tr_encoded["Full_Text"] = le.fit_transform(tr_encoded["Full_Text"])

test_encoded = test_data_df.copy()
test_encoded["Party"] = le.fit_transform(test_encoded["Party"])
test_encoded["Tweet_Text"] = le.fit_transform(test_encoded["Tweet_Text"])
#test_encoded["Hashtags"] = le.fit_transform(test_encoded["Hashtags"])
#test_encoded["Mentions"] = le.fit_transform(test_encoded["Mentions"])
test_encoded["Retweet"] = le.fit_transform(test_encoded["Retweet"])
test_encoded["Handle"] = le.fit_transform(test_encoded["Handle"])
test_encoded["Full_Text"] = le.fit_transform(test_encoded["Full_Text"])


#x_train = tr_data_df[["Char_Count", "Num_Hashtags", "Num_Mentions", "Retweet"]]
y_train = tr_data_df[["Party"]]

#x_test = test_data_df[["Char_Count", "Num_Hashtags", "Num_Mentions", "Retweet"]]
y_test = test_data_df[["Party"]]

#x_en_train = tr_encoded[["Tweet_Text", "Hashtags" "Mentions", "Retweet", "Handle", "Full_Text"]]
x_en_train = tr_encoded[["Tweet_Text", "Retweet", "Handle", "Full_Text"]]
y_en_train = tr_encoded[["Party"]]

#x_en_test = test_encoded[["Tweet_Text", "Hashtags", "Mentions", "Retweet", "Handle", "Full_Text"]]
x_en_test = test_encoded[["Tweet_Text", "Retweet", "Handle", "Full_Text"]]
y_en_test = test_encoded[["Party"]]

In [105]:
x_en_train

,Tweet_Text,Retweet,Handle,Full_Text
0,49548,0,138,52547
1,11170,1,138,39760
2,885,1,138,35608
3,5863,1,138,35570
4,8498,1,138,32523
...,...,...,...,...
59040,25744,0,414,17731
59041,4428,0,414,276
59042,54931,0,414,56904
59043,5920,0,414,253


In [106]:
y_train

,Party
0,Democrat
1,Democrat
2,Democrat
3,Democrat
4,Democrat
...,...
59040,Republican
59041,Republican
59042,Republican
59043,Republican


In [107]:
y_en_train

,Party
0,0
1,0
2,0
3,0
4,0
...,...
59040,1
59041,1
59042,1
59043,1


In [108]:
x_en_test

,Tweet_Text,Retweet,Handle,Full_Text
0,10282,0,107,215
1,4994,0,123,2942
2,5091,0,415,3033
3,1841,1,286,9865
4,8715,0,58,6497
...,...,...,...,...
14756,13611,1,414,14052
14757,7045,0,22,4901
14758,4073,0,309,2080
14759,1865,1,149,8007


In [109]:
y_test

,Party
0,Democrat
1,Republican
2,Republican
3,Democrat
4,Democrat
...,...
14756,Republican
14757,Democrat
14758,Democrat
14759,Democrat


In [110]:
y_en_test

,Party
0,0
1,1
2,1
3,0
4,0
...,...
14756,1
14757,0
14758,0
14759,0


In [111]:
# Build ID3 tree using entropy
clf = tree.DecisionTreeClassifier(criterion="entropy")
clf.fit(x_en_train, y_en_train)

# Display Tree
#dot_data = tree.export_graphviz(clf, out_file=None,
                                #feature_names=["Tweet_Text", "Retweet", "Handle", "Full_Text"],
                                #class_names="DR",
                                #filled=True, rounded=True,
                                #special_characters=True)  
#graph = graphviz.Source(dot_data)
#graph

DecisionTreeClassifier(criterion='entropy')

In [112]:
# Print Accuracy
y_pred = clf.predict(x_en_test)
print("ID3 Tree Results")
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_en_test, y_pred))

ID3 Tree Results
Predicted: [0 1 1 ... 0 0 1]
Accuracy: 0.9983740938960775


In [113]:
# Build tree using Gini
clf = tree.DecisionTreeClassifier()
clf.fit(x_en_train, y_en_train)

# Display Tree
#dot_data = tree.export_graphviz(clf, out_file=None,
                                #feature_names=["Tweet_Text", "Retweet", "Handle", "Full_Text"],
                                #class_names="DR",
                                #filled=True, rounded=True,
                                #special_characters=True)  
#graph = graphviz.Source(dot_data)
#graph

DecisionTreeClassifier()

In [114]:
# Print Accuracy
y_pred = clf.predict(x_en_test)
print("Gini Tree Results")
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_en_test, y_pred))

Gini Tree Results
Predicted: [0 1 1 ... 0 0 1]
Accuracy: 0.9992547930357022


In [115]:
# Random Forest
#clf = RandomForestClassifier(max_depth=7, min_samples_split=0.1, min_samples_leaf=0.05)
#clf.fit(x_en_train, y_en_train)
#y_pred = clf.predict(x_en_test)
#print("Random Forest Results")
#print("Predicted:", y_pred)
#print("Accuracy:", metrics.accuracy_score(y_en_test, y_pred))